<a href="https://colab.research.google.com/github/MaxGhenis/random/blob/master/warren_tax_avoidance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tax avoidance in Warren and Sanders wealth taxes

Compare the current 16% avoidance to an 8% per 1% elasticity.

For example, model 24% avoidance for families with net worth over \$1 billion who would have a 3% marginal wealth tax rate under the Warren plan, and up to 64% avoidance for families with over \$10 billion net worth for Sanders plan.

## Setup

TODO: Figure out how to make microdf work without installing labellines and taxcalc (optional).

In [0]:
import sys

In [0]:
!pip install matplotlib-label-lines
if 'microdf' not in sys.modules:
    !pip install git+git://github.com/maxghenis/microdf.git
if 'taxcalc' not in sys.modules:
    !pip install git+git://github.com/PSLmodels/Tax-Calculator.git

In [0]:
import pandas as pd
import numpy as np
import microdf as mdf

## Load data

From Saez/Zucman wealth tax calculator: https://github.com/BITSS/opa-wealthtax

In [0]:
wealth = pd.read_stata('https://github.com/BITSS/opa-wealthtax/blob/master/analysis_data/wealth.dta?raw=true')

## Descriptive analysis

Total net worth

In [5]:
total_nw = mdf.weighted_sum(wealth, 'networth', 'weight')
total_nw / 1e12

93.74638072480977

In [6]:
total_families = wealth.weight.sum()
total_families / 1e6

156.43578

### Warren wealth tax

2% above \$50 million, 3% above \$1 billion.

16% avoidance/evasion rate per Saez and Zucman, which is 2% * elasticity of 8: http://wealthtaxsimulator.org/analysis/

In [0]:
AVOID_ELASTICITY = 8
CONST_AVOID_RATE = 0.16  # Originally computed as 8 * 0.02.

In [0]:
def tax(val, brackets, rates):
    # Args:
    #     val: Value to assess tax on, e.g. wealth or income (list or Series).
    #     brackets: Left side of each bracket (list or Series).
    #     rates: Rate corresponding to each bracket.
    df_tax = pd.DataFrame({'brackets': brackets, 'rates': rates})
    df_tax['base_tax'] = df_tax.brackets.\
        sub(df_tax.brackets.shift(fill_value=0)).\
        mul(df_tax.rates.shift(fill_value=0)).cumsum()
    rows = df_tax.brackets.searchsorted(val, side='right') - 1
    income_bracket_df = df_tax.loc[rows].reset_index(drop=True)
    return pd.Series(val).sub(income_bracket_df.brackets).\
        mul(income_bracket_df.rates).add(income_bracket_df.base_tax)

In [0]:
def mtr(val, brackets, rates):
    # Args:
    #     val: Value to assess tax on, e.g. wealth or income (list or Series).
    #     brackets: Left side of each bracket (list or Series).
    #     rates: Rate corresponding to each bracket.
    df_tax = pd.DataFrame({'brackets': brackets, 'rates': rates})
    df_tax['base_tax'] = df_tax.brackets.\
        sub(df_tax.brackets.shift(fill_value=0)).\
        mul(df_tax.rates.shift(fill_value=0)).cumsum()
    rows = df_tax.brackets.searchsorted(val, side='right') - 1
    income_bracket_df = df_tax.loc[rows].reset_index(drop=True)
    return income_bracket_df.rates

In [0]:
WARREN_RATES = [0, 0.02, 0.03]  # 0%, 2%, 3%.

WARREN_BRACKETS = [0,
                   50e6,  # First $50 million.
                   1e9]   # Over $1 billion.

In [0]:
def warren_tax(incomes):
    return tax(incomes, rates=WARREN_RATES, brackets=WARREN_BRACKETS)

In [0]:
wealth['warren_mtr'] = mtr(np.maximum(wealth.networth, 0),
                           WARREN_BRACKETS, WARREN_RATES)

### Current approach

16% constant avoidance.

In [0]:
wealth['warren_tax_base'] = ((1 - CONST_AVOID_RATE) * 
                             np.maximum(wealth.networth, 0))

In [0]:
wealth['warren_tax'] = warren_tax(wealth.warren_tax_base)
wealth['networth_warren'] = wealth.networth - wealth.warren_tax

In [15]:
warren_revenue = (mdf.weighted_sum(wealth, 'networth', 'weight') - 
                  mdf.weighted_sum(wealth, 'networth_warren', 'weight')).sum()
warren_revenue / 1e9

199.15986065067187

### Adjusted approach

In [0]:
wealth['warren_tax_base2'] = ((1 - AVOID_ELASTICITY * wealth.warren_mtr) * 
                              np.maximum(wealth.networth, 0))

In [0]:
wealth['warren_tax2'] = warren_tax(wealth.warren_tax_base2)
wealth['networth_warren2'] = wealth.networth - wealth.warren_tax2

In [18]:
warren_revenue2 = (mdf.weighted_sum(wealth, 'networth', 'weight') - 
                   mdf.weighted_sum(wealth, 'networth_warren2', 'weight')).sum()
warren_revenue2 / 1e9

189.69935268021874

In [19]:
warren_revenue2 / warren_revenue

0.9524979183077109

## Sanders wealth tax

In [0]:
SANDERS_RATES =    [0, 0.01, 0.02,  0.03,  0.04, 0.05,  0.06, 0.07, 0.08]
SANDERS_BRACKETS = [0, 32e6, 50e6, 250e6, 500e6,  1e9, 2.5e9,  5e9, 10e9]

In [0]:
wealth['sanders_mtr'] = mtr(np.maximum(wealth.networth, 0),
                            SANDERS_BRACKETS, SANDERS_RATES)

In [0]:
def sanders_tax(incomes):
    return tax(incomes, rates=SANDERS_RATES, brackets=SANDERS_BRACKETS)

### Current approach

16% constant avoidance.

In [0]:
wealth['sanders_tax_base'] = ((1 - CONST_AVOID_RATE) * 
                              np.maximum(wealth.networth, 0))

In [0]:
wealth['sanders_tax'] = sanders_tax(wealth.sanders_tax_base)
wealth['networth_sanders'] = wealth.networth - wealth.sanders_tax

In [25]:
sanders_revenue = (mdf.weighted_sum(wealth, 'networth', 'weight') - 
                   mdf.weighted_sum(wealth, 'networth_sanders', 'weight')).sum()
sanders_revenue / 1e9

324.3443422851719

### Adjusted approach

In [0]:
wealth['sanders_tax_base2'] = ((1 - AVOID_ELASTICITY * wealth.sanders_mtr) *
                               np.maximum(wealth.networth, 0))

In [0]:
wealth['sanders_tax2'] = sanders_tax(wealth.sanders_tax_base2)
wealth['networth_sanders2'] = wealth.networth - wealth.sanders_tax2

In [28]:
sanders_revenue2 = (mdf.weighted_sum(wealth, 'networth', 'weight') - 
                   mdf.weighted_sum(wealth, 'networth_sanders2', 'weight')).sum()
sanders_revenue2 / 1e9

215.8235250503125

In [29]:
sanders_revenue2 / sanders_revenue

0.6654147981423857